<a href="https://colab.research.google.com/github/faisal12212092/DDoS-Attack-Detection-Using-Deep-Learning-Research-report-/blob/main/Hybridmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Bidirectional, LSTM,Dropout
from tensorflow.keras.optimizers import Adam



In [ ]:
path = "/content/drive/MyDrive/DDOS DATASETS/smtnormalizeddata.csv"
df = pd.read_csv(path)

In [ ]:
df.dropna(inplace=True)
features = df.drop('Label', axis=1)
target = df['Label']

In [ ]:
def create_sequences(data, labels, sequence_length):
    sequences = []
    sequence_labels = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        sequence_labels.append(labels[i + sequence_length])
    return np.array(sequences), np.array(sequence_labels)


In [ ]:
sequence_length = 10

# Create sequences
X, y = create_sequences(features.values, target.values, sequence_length)

# Define K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

cv_accuracy = []
cv_precision = []
cv_recall = []
cv_f1 = []
cv_cm = []

# K-fold cross-validation
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = Sequential()
    model.add(SimpleRNN(32, return_sequences=True, input_shape=(sequence_length, X_train.shape[2])))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2)

Epoch 1/5
1281/1281 [==============================] - 30s 19ms/step - loss: 0.0144 - accuracy: 0.9966 - val_loss: 0.0037 - val_accuracy: 0.9992
Epoch 2/5
1281/1281 [==============================] - 21s 16ms/step - loss: 0.0046 - accuracy: 0.9990 - val_loss: 0.0054 - val_accuracy: 0.9985
Epoch 3/5
1281/1281 [==============================] - 20s 16ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.0036 - val_accuracy: 0.9992
Epoch 4/5
1281/1281 [==============================] - 22s 17ms/step - loss: 0.0035 - accuracy: 0.9993 - val_loss: 0.0027 - val_accuracy: 0.9995
Epoch 5/5
1281/1281 [==============================] - 20s 16ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.0022 - val_accuracy: 0.9996
Epoch 1/5
1281/1281 [==============================] - 27s 18ms/step - loss: 0.0151 - accuracy: 0.9965 - val_loss: 0.0032 - val_accuracy: 0.9991
Epoch 2/5
1281/1281 [==============================] - 21s 16ms/step - loss: 0.0045 - accuracy: 0.9991 - val_loss: 0.0015 - val_ac

In [ ]:
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)

    # Compute metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)


1601/1601 [==============================] - 8s 5ms/step


In [ ]:
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    cv_f1.append(f1)
    cv_cm.append(cm)

avg_accuracy = np.mean(cv_accuracy)
avg_precision = np.mean(cv_precision)
avg_recall = np.mean(cv_recall)
avg_f1 = np.mean(cv_f1)
avg_cm = np.mean(cv_cm, axis=0)

# Print average metrics and confusion matrix
print(f'Average Accuracy: {avg_accuracy}')
print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')
print(f'Average F1 Score: {avg_f1}')
print('Average Confusion Matrix:')
print(avg_cm.astype(int))

Average Accuracy: 0.9991798156538041
Average Precision: 0.9996875488204968
Average Recall: 0.9986734295747172
Average F1 Score: 0.999180231877269
Average Confusion Matrix:
[[25570     8]
 [   34 25596]]
